In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

In [3]:
import os
import pandas as pd
import re
import requests
from io import BytesIO
from PIL import Image, ImageEnhance
import random
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import pickle
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from sklearn.metrics.pairwise import cosine_similarity
import requests
from PIL import Image
from io import BytesIO
import ast

In [4]:
folder_path ='/content/drive/MyDrive/IR dataset/Assignment-2'
os.chdir(folder_path)

In [5]:
# Load CSV data
csv_file_path = 'A2_Data.csv'
df = pd.read_csv(csv_file_path)

# Rename the "Unnamed: 0" column to "ProductID"
df = df.rename(columns={"Unnamed: 0": "ProductID"})
df


,ProductID,Image,Review Text
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...
...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the..."


In [6]:
# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to extract features using VGG16
def extract_features(img):
    features = vgg_model.predict(img)
    flattened_features = features.flatten()
    return flattened_features / np.linalg.norm(flattened_features)  # Normalize the features


In [7]:
# Load the VGG16 model pre-trained on ImageNet dataset
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [8]:

# Function to fetch and preprocess input image from URL
def preprocess_input_image_url(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content)).resize((224, 224))  # Resize the image
    img_path = "temp_input_image.jpg"  # Temporary file path
    img.save(img_path)  # Save the input image temporarily
    img = preprocess_image(img_path)
    os.remove(img_path)  # Remove the temporary input image file
    return img

In [9]:
from PIL import Image, UnidentifiedImageError

In [10]:
# Preprocess and extract features for all images
image_features = {}
for index, row in df.iterrows():
    img_urls = ast.literal_eval(row['Image'])  # Convert the string list to a Python list
    img_url = img_urls[0]  # Assuming each row contains only one image URL
    response = requests.get(img_url)
    try:
        img = Image.open(BytesIO(response.content)).resize((224, 224))  # Resize the image
        img_path = f"temp_image_{index}.jpg"  # Temporary file path
        img.save(img_path)  # Save the image temporarily
        img = preprocess_image(img_path)
        image_features[index] = extract_features(img)
        os.remove(img_path)  # Remove the temporary image file
    except (IOError, OSError, UnidentifiedImageError) as e:
        print(f"Error processing image at index {index}: {e}")

# Save the image features dictionary to a pickle file
with open('image_features.pkl', 'wb') as f:
    pickle.dump(image_features, f)

1/1 [==============================] - 1s 530ms/step
Error processing image at index 67: cannot identify image file <_io.BytesIO object at 0x7fd461f0d0d0>
1/1 [==============================] - 1s 774ms/step
Error processing image at index 110: cannot identify image file <_io.BytesIO object at 0x7fd461f26610>
1/1 [==============================] - 1s 555ms/step
Error processing image at index 523: cannot identify image file <_io.BytesIO object at 0x7fd4612e3920>
1/1 [==============================] - 1s 902ms/step
Error processing image at index 701: cannot identify image file <_io.BytesIO object at 0x7fd461b30540>
1/1 [==============================] - 1s 683ms/step
Error processing image at index 860: cannot identify image file <_io.BytesIO object at 0x7fd461aae840>
1/1 [==============================] - 1s 535ms/step
Error processing image at index 936: cannot identify image file <_io.BytesIO object at 0x7fd461dafb00>
1/1 [==============================] - 1s 538ms/step


In [11]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Lower-Casing
        text = text.lower()

        # Remove ellipses ("...") and words with apostrophes attached ("'m", "'ve", etc.)
        text = re.sub(r'\.\.\.', '', text)
        text = re.sub(r'\s*\'[a-z]*\s*', ' ', text)

        # Tokenization
        tokens = nltk.word_tokenize(text)

        # Remove stopwords and punctuation
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation]

        # Remove blank space tokens
        tokens = [token for token in tokens if token.strip() != '']

        # Stemming
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        return tokens
    else:
        return []  # Return an empty list for non-string values

In [13]:
# Apply pre-processing to the 'Review Text' column
df['ProcessedReviewText'] = df['Review Text'].apply(preprocess_text)

# Display the DataFrame with the processed text
print(df[['ProductID', 'ProcessedReviewText']].head())

   ProductID                                ProcessedReviewText
0       3452  [love, vintag, spring, vintag, strat, good, te...
1       1205  [work, great, guitar, bench, mat, rug, enough,...
2       1708  [use, everyth, acoust, bass, ukulel, know, sma...
3       2078  [great, price, good, qualiti, quit, match, rad...
4        801  [bought, bass, split, time, primari, bass, dea...


In [14]:
df


,ProductID,Image,Review Text,ProcessedReviewText
0,3452,['https://images-na.ssl-images-amazon.com/imag...,Loving these vintage springs on my vintage str...,"[love, vintag, spring, vintag, strat, good, te..."
1,1205,['https://images-na.ssl-images-amazon.com/imag...,Works great as a guitar bench mat. Not rugged ...,"[work, great, guitar, bench, mat, rug, enough,..."
2,1708,['https://images-na.ssl-images-amazon.com/imag...,We use these for everything from our acoustic ...,"[use, everyth, acoust, bass, ukulel, know, sma..."
3,2078,['https://images-na.ssl-images-amazon.com/imag...,Great price and good quality. It didn't quite...,"[great, price, good, qualiti, quit, match, rad..."
4,801,['https://images-na.ssl-images-amazon.com/imag...,I bought this bass to split time as my primary...,"[bought, bass, split, time, primari, bass, dea..."
...,...,...,...,...
995,1265,['https://images-na.ssl-images-amazon.com/imag...,Extremely impressed with this kit.,"[extrem, impress, kit]"
996,1882,['https://images-na.ssl-images-amazon.com/imag...,This is a great stereo reverb with plenty of c...,"[great, stereo, reverb, plenti, control, get, ..."
997,1547,['https://images-na.ssl-images-amazon.com/imag...,I really like the simplicity of this bridge. I...,"[realli, like, simplic, bridg, adjust, easi, s..."
998,1004,['https://images-na.ssl-images-amazon.com/imag...,"Great Product, but there is no warranty in the...","[great, product, warranti, usa, purchas, amazo..."


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the tokenized words back into sentences
df['ProcessedReviewText'] = df['ProcessedReviewText'].apply(lambda x: ' '.join(x))

# Calculate TF-IDF scores
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['ProcessedReviewText'])

# Convert TF-IDF matrix to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Display the DataFrame with TF-IDF scores
print(tfidf_df.head())


    00  000  0000  000hz   02  022  024   05   07   09  ...  yuk  zakk  zelda  \
0  0.0  0.0   0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   0.0    0.0   
1  0.0  0.0   0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   0.0    0.0   
2  0.0  0.0   0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   0.0    0.0   
3  0.0  0.0   0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   0.0    0.0   
4  0.0  0.0   0.0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   0.0    0.0   

   zero  zildjian  zip  ziplock  zipper  zoom  zuyoagubnri  
0   0.0       0.0  0.0      0.0     0.0   0.0          0.0  
1   0.0       0.0  0.0      0.0     0.0   0.0          0.0  
2   0.0       0.0  0.0      0.0     0.0   0.0          0.0  
3   0.0       0.0  0.0      0.0     0.0   0.0          0.0  
4   0.0       0.0  0.0      0.0     0.0   0.0          0.0  

[5 rows x 4781 columns]


In [ ]:
# Assuming tfidf_df is your DataFrame
if (tfidf_df != 0).any().any():
    print("There are non-zero values in the DataFrame.")
    non_zero_values = tfidf_df[tfidf_df != 0].stack()
    print("Non-zero values:")
    print(non_zero_values)
else:
    print("All values in the DataFrame are zero.")


There are non-zero values in the DataFrame.
Non-zero values:
0    bridg         0.209655
     float         0.314115
     go            0.147586
     good          0.124059
     great         0.102781
                     ...   
999  product       0.215060
     profession    0.363204
     ship          0.337541
     use           0.149399
     worth         0.296662
Length: 32689, dtype: float64


In [16]:
# Save the extracted image features using pickle
image_features_path = os.path.join(folder_path, 'image_features.pkl')
with open(image_features_path, 'wb') as f:
    pickle.dump(image_features, f)

# Save the TF-IDF scores using pickle
tfidf_scores_path = os.path.join(folder_path, 'tfidf_scores.pkl')
with open(tfidf_scores_path, 'wb') as f:
    pickle.dump(tfidf_df, f)

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [18]:
# Placeholder for image and review URLs
input_image_url = 'https://images-na.ssl-images-amazon.com/images/I/81NKeeOsWAL._SY88.jpg'
input_review = "It works! This is a nice product I'm happy, however there are some things I noticed but nothing to take much away from the pedal, especially considering the low price."


In [19]:
def find_most_similar_images(input_features, all_image_features, top_k=3):
    # Calculate cosine similarity between input features and all other image features
    similarities = {}
    for idx, features in all_image_features.items():
        similarities[idx] = cosine_similarity([input_features], [features])[0][0]

    # Sort the similarities dictionary by values
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

    # Get the top k most similar images
    top_similar_images = sorted_similarities[:top_k]
    return top_similar_images


In [20]:
# Find the most similar reviews based on TF-IDF scores
def find_most_similar_reviews(input_tfidf_scores, all_tfidf_scores, top_k=3):
    # Filter out missing or null values from all_tfidf_scores
    valid_tfidf_scores = [scores for scores in all_tfidf_scores if scores is not None]

    # Check if valid_tfidf_scores is empty
    if not valid_tfidf_scores:
        print("Error: All TF-IDF scores are missing or null.")
        return None, None

    similarities = cosine_similarity(input_tfidf_scores, valid_tfidf_scores)[0]
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return top_indices, similarities[top_indices]

In [21]:
# Preprocess the input review text
preprocessed_input_review = preprocess_text(input_review)

# Display or use the preprocessed input review as needed
print("Preprocessed Input Review:")
print(preprocessed_input_review)

Preprocessed Input Review:
['work', 'nice', 'product', 'happi', 'howev', 'thing', 'notic', 'noth', 'take', 'much', 'away', 'pedal', 'especi', 'consid', 'low', 'price']


In [22]:
# Calculate TF-IDF scores for the preprocessed input review
input_tfidf_scores = tfidf_vectorizer.transform([' '.join(preprocessed_input_review)]).toarray()

In [23]:
input_img = preprocess_input_image_url(input_image_url)

# Extract features for the input image
input_features = extract_features(input_img)

1/1 [==============================] - 1s 539ms/step


In [24]:
# Find the most similar reviews based on TF-IDF scores
top_review_indices, review_similarities = find_most_similar_reviews(input_tfidf_scores, tfidf_df.values, top_k=3)


In [25]:
# Find the most similar images based on image features
top_similar_images = find_most_similar_images(input_features, image_features, top_k=3)


In [26]:
# Function to compute composite similarity score
def compute_composite_similarity(image_sim, text_sim):
    return 0.5 * image_sim + 0.5 * text_sim

In [27]:
print("USING IMAGE RETRIEVAL")

for i, (idx, similarity) in enumerate(top_similar_images, 1):
    image_urls = eval(df.loc[idx, 'Image'])
    review_text = df.loc[idx, 'Review Text']

    # Reshape the TF-IDF scores array to 2D
    tfidf_scores_2d = tfidf_df.values[idx].reshape(1, -1)

    # Calculate cosine similarity of text
    _, text_similarity = find_most_similar_reviews(input_tfidf_scores, tfidf_scores_2d, top_k=1)
    text_similarity_value = text_similarity[0]  # Extracting the value from the array

    # Compute composite similarity score
    composite_similarity = compute_composite_similarity(similarity, text_similarity_value)

    print(f"{i}) Image URL: {image_urls}")
    print(f"   Review: {review_text}")
    print(f"   Cosine similarity of images - {similarity:.4f}")
    print(f"   Cosine similarity of text - {text_similarity_value:.4f}")  # Format the extracted value
    print(f"   Composite similarity score: {composite_similarity:.4f}")
    print()


USING IMAGE RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81NKeeOsWAL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81Ogy5-bqZL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81VcxW9hGkL._SY88.jpg']
   Review: It works! This is a nice product I'm happy, however there are some things I noticed but nothing to take much away from the pedal, especially considering the low price.

Pros:

(out of 5 ratings)

*Great price: 5/5
*Sturdy build: 5/5
*Tunes accurately: 5/5 (I used this along with a reliable tuner app and they were both tuning simultaneously)
*Size and looks 5/5

Cons: (some people may consider these as cons)

(out of 5 rating)

*Tuning speed: 3/5 (you have to hit the string once to it for fastest: repeatedly hitting the string takes longer)
*Power source: 4/5 (Doesn't take battery, but any 9v adapter will work)

Overall product rating: 4/5
   Cosine similarity of images - 1.0000
   Cosine similarity of text - 0.4859
   Co

In [28]:
print("USING TEXT RETRIEVAL")

for i, idx in enumerate(top_review_indices, 1):
    review_text = df.loc[idx, 'Review Text']
    corresponding_image_urls = eval(df.loc[idx, 'Image'])

    # Reshape the TF-IDF scores array to 2D
    tfidf_scores_2d = tfidf_df.iloc[idx].values.reshape(1, -1)

    # Calculate cosine similarity of text
    _, text_similarity = find_most_similar_reviews(input_tfidf_scores, tfidf_scores_2d, top_k=1)
    text_similarity_value = text_similarity[0]  # Extracting the value from the array

    # Compute composite similarity score
    composite_similarity = compute_composite_similarity(text_similarity_value, review_similarities[i-1])

    print(f"{i}) Image URL: {corresponding_image_urls}")
    print(f"   Review: {review_text}")
    print(f"   Cosine similarity of images - {review_similarities[i-1]:.4f}")
    print(f"   Cosine similarity of text - {text_similarity_value:.4f}")
    print(f"   Composite similarity score: {composite_similarity:.4f}")
    print()


USING TEXT RETRIEVAL
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81NKeeOsWAL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81Ogy5-bqZL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81VcxW9hGkL._SY88.jpg']
   Review: It works! This is a nice product I'm happy, however there are some things I noticed but nothing to take much away from the pedal, especially considering the low price.

Pros:

(out of 5 ratings)

*Great price: 5/5
*Sturdy build: 5/5
*Tunes accurately: 5/5 (I used this along with a reliable tuner app and they were both tuning simultaneously)
*Size and looks 5/5

Cons: (some people may consider these as cons)

(out of 5 rating)

*Tuning speed: 3/5 (you have to hit the string once to it for fastest: repeatedly hitting the string takes longer)
*Power source: 4/5 (Doesn't take battery, but any 9v adapter will work)

Overall product rating: 4/5
   Cosine similarity of images - 0.4859
   Cosine similarity of text - 0.4859
   Com

In [29]:
# Initialize lists to store composite similarity scores and pairs
composite_similarity_scores = []
pairs = []

In [30]:
# Create a list to store the pairs with their composite similarity scores
combined_pairs = []

# Iterate over the top similar images obtained from image retrieval
for idx, image_similarity in top_similar_images:
    image_urls = eval(df.loc[idx, 'Image'])
    review_text = df.loc[idx, 'Review Text']

    # Reshape the TF-IDF scores array to 2D
    tfidf_scores_2d = tfidf_df.values[idx].reshape(1, -1)

    # Calculate cosine similarity of text
    _, text_similarity = find_most_similar_reviews(input_tfidf_scores, tfidf_scores_2d, top_k=1)
    text_similarity_value = text_similarity[0]  # Extracting the value from the array

    # Compute composite similarity score (average)
    composite_similarity = (image_similarity + text_similarity_value) / 2

    # Add the pair and composite similarity score to the list
    combined_pairs.append((idx, image_similarity, text_similarity_value, composite_similarity))

# Iterate over the top similar reviews obtained from text retrieval
for idx in top_review_indices:
    review_text = df.loc[idx, 'Review Text']
    corresponding_image_urls = eval(df.loc[idx, 'Image'])

    # Calculate cosine similarity of images (previously calculated)
    image_similarity = review_similarities[i-1]

    # Compute composite similarity score (average)
    composite_similarity = (image_similarity + text_similarity_value) / 2

    # Add the pair and composite similarity score to the list
    combined_pairs.append((idx, image_similarity, text_similarity_value, composite_similarity))

# Rank the pairs based on the composite similarity score
ranked_pairs = sorted(combined_pairs, key=lambda x: x[3], reverse=True)

In [31]:
# Print the ranked pairs
print("Combined Retrieval (Text and Image) - Ranked Pairs:")
for i, (idx, image_similarity, text_similarity, composite_similarity) in enumerate(ranked_pairs, 1):
    found_in_image_retrieval = False
    for top_idx, _ in top_similar_images:
        if idx == top_idx:
            found_in_image_retrieval = True
            break

    if found_in_image_retrieval:  # Check if the pair is from image retrieval
        image_urls = eval(df.loc[idx, 'Image'])
        review_text = df.loc[idx, 'Review Text']
    else:  # If the pair is from text retrieval
        image_urls = eval(df.loc[idx, 'Image'])
        review_text = df.loc[idx, 'Review Text']

    print(f"{i}) Image URL: {image_urls}")
    print(f"   Review: {review_text}")
    print(f"   Cosine similarity of images: {image_similarity:.4f}")
    print(f"   Cosine similarity of text: {text_similarity:.4f}")
    print(f"   Composite similarity score: {composite_similarity:.4f}")
    print()


Combined Retrieval (Text and Image) - Ranked Pairs:
1) Image URL: ['https://images-na.ssl-images-amazon.com/images/I/81NKeeOsWAL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81Ogy5-bqZL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81VcxW9hGkL._SY88.jpg']
   Review: It works! This is a nice product I'm happy, however there are some things I noticed but nothing to take much away from the pedal, especially considering the low price.

Pros:

(out of 5 ratings)

*Great price: 5/5
*Sturdy build: 5/5
*Tunes accurately: 5/5 (I used this along with a reliable tuner app and they were both tuning simultaneously)
*Size and looks 5/5

Cons: (some people may consider these as cons)

(out of 5 rating)

*Tuning speed: 3/5 (you have to hit the string once to it for fastest: repeatedly hitting the string takes longer)
*Power source: 4/5 (Doesn't take battery, but any 9v adapter will work)

Overall product rating: 4/5
   Cosine similarity of images: 1.0000
   Cosine simi